# Auto-Regressive assimilation with EnIF

We illustrate `EnIF` methodology and the API through the Auto-Regressive-1 (AR-1) model.
The AR-1 is defined by
$$u_t = \phi u_{t-1} + \epsilon,~~
 \epsilon\sim N(0,1) $$
and has a tri-diagonal sparse precision matrix.

We will illustrate both the **high-level** API of EnIF, and also the **low-level** API.

## Imports

In [1]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import scipy as sp
from scipy.stats import norm
import matplotlib.pyplot as plt
from graphite_maps.precision_estimation import precision_to_graph
from graphite_maps.enif import EnIF

We will repeatedly plot
- a realization
- an ensemble
- the ensemble mean
- the difference between posterior and prior mean

we thus create a function for this

In [ ]:
COLORS = plt.rcParams['axes.prop_cycle'].by_key()['color']

def plot_ensemble(Ensemble, d, prior_mean=None):
    n, p = Ensemble.shape
    u_position = [j for j in range(p)]
    ind_obs = np.rint(p / 2).astype(int) - 1
    ensemble_mean = np.array(Ensemble.mean(axis=0))
    num_plots = 3 if prior_mean is None else 4
    
    fig = plt.figure(figsize=(16, 4))
    fig.suptitle('Ensemble inspection & observation', fontsize=16)

    ax = fig.add_subplot(1,num_plots,1)
    ax.plot(u_position, Ensemble[0,:])
    ax.plot(ind_obs, [d], marker='*', ls='none', ms=20)
    ax.set_title('Realization #1')
    
    ax = fig.add_subplot(1,num_plots,2)
    for i in range(n):
        ax.plot(u_position, Ensemble[i,:], label= "realization {ind}".format(ind=i), color=COLORS[0], alpha=0.1)
    ax.set_title('All realizations')
    ax.plot(ind_obs, [d], marker='*', ls='none', ms=20, color=COLORS[1])
    ax_min, ax_max = ax.get_ylim()  # Get y-limits
    
    ax = fig.add_subplot(1,num_plots,3)
    ax.plot(u_position, ensemble_mean)
    ax.plot(ind_obs, [d], marker='*', ls='none', ms=20)
    ax.set_title('Ensemble mean')

    if num_plots == 4:
        ax = fig.add_subplot(1,num_plots,4)
        ax.plot(u_position, ensemble_mean - prior_mean)
        ax.set_title('Posterior mean - prior mean')
    plt.show()

## Data setup: Auto-regressive simulation

We define the parameters of the ensemble based data assimilation with the AR-1
- `n`: ensemble or sample size. Increasing will increase the quality of the solution
- `p`: the number of AR-1 time-steps, here the dimensions of the problem. Increasing yields a more difficult problem. Traditional ensemble based methods would use localization to deal with high `p`.
- `phi`: between `-1` and `1`. `0` yields white noise without dependence. When `abs(phi)` is close to `1` there is strong dependence between dimensions.
- - Suggest to try `phi in [-0.99, 0.5, 0, 0.5, 0.99]` and inspect differences.

In [ ]:
# Define parameters
n = 100  # Sample size
p = 1000   # Dimension
phi = 0.99

In [ ]:
# Simulate data
def rar1(T, phi):
    """simulate auto-regressive-1.
    The first element is simulated from stationary distribution.
    """
    x = np.empty([T])
    x[0] = np.random.normal(0, 1 / np.sqrt(1 - phi**2))
    for i in range(1, T):
        x[i] = phi * x[i - 1] + np.random.normal(0, 1)
    return x

np.random.seed(42)
U = np.array([rar1(T=p, phi=phi) for _ in range(n)])

We here know the precision exactly, so we may compare our estimates. Note we derive the graph `Graph_u` from the precision, which we would normally just know apriori, while `Prec_u` would often (not always) be unknown.

In [ ]:
# create AR-1 precision matrix
Prec_u = sp.sparse.diags(
    [np.repeat(-phi, p-1), np.concatenate(([1.0], np.repeat(1.0+phi**2, p-2), [1.0])), np.repeat(-phi, p-1)], 
    [-1,0,1], shape=(p, p), 
    format='csc'
)

# create corresponding graph -- often we only know this
Graph_u = precision_to_graph(Prec_u)

Create observations, `d`, that are associated with `u`.
We here pass a linear map `H` so that we model $d=Hu+\epsilon$, and $\epsilon$ has known precision.
Note that `H` may be learnt from data using the `fit_H` function.
We must then pass realizations `U` and simulated responses `Y`.

In [ ]:
# create observation
d = np.array([30.0])
sd_eps = 1
H = np.array([0] * p, ndmin=2)
H[0, np.rint(p / 2).astype(int) - 1] = 1  # middle sencor
H = sp.sparse.csc_matrix(H)
Prec_eps = np.array([1/sd_eps**2], ndmin=2)
Prec_eps = sp.sparse.csc_matrix(Prec_eps)

In [ ]:
# create responses
Y = U@H.T

Let's inspect the sample from the prior, and the observation `d`

In [ ]:
plot_ensemble(U, d)

## EnKF / Ensemble Smoother

In [ ]:
def ES(U, Y, d, Cov_epsilon):
    # Cross-covariance matrix C_UY has shape (p, m)
    n, m = Y.shape
    C_UY = np.dot((U - U.mean(axis=0)).T, (Y - Y.mean(axis=0))) / (n - 1)
    print(C_UY.shape)

    # Compute the covariance of Y
    # Covariance matrix C_YY has shape (m, m)
    C_YY = np.dot((Y - Y.mean(axis=0)).T, (Y - Y.mean(axis=0))) / (n - 1)

    print(C_YY.shape)

    # Estimated K
    K = C_UY @ np.linalg.inv(C_YY + Cov_epsilon)
    print(K.shape)
    
    # perturb observations
    n, m = Y.shape
    eps = np.random.normal(loc=0, scale=np.sqrt(Cov_epsilon.diagonal()), size=(n,m))
    di = d - eps

    U_posterior = np.copy(U)
    for i in range(n):
        U_posterior[i,:] += K @ (di[i] - Y[i,:])
    
    return U_posterior

In [ ]:
Y = U@H.T
Y.shape

In [ ]:
U_posterior_es = ES(U, Y, d, np.linalg.inv(Prec_eps.A))

In [ ]:
prior_mean = np.array(U.mean(axis=0))
plot_ensemble(U_posterior_es, d, prior_mean)

## EnIF high-level API

The high-level API consists of initialization, usually with a `Graph_u`, the noise precision `Prec_eps` and possibly the map `H`.
Then only two functions are needed `fit` and `transport` to arrive an updated (or analysed) ensemble.
To think of these know that 
- `fit` estimates a GMRF model w.r.t. the graph `Graph_u`, and learns a sparse affine map `H`. If `Prec_u` or `H` are passed, they are not re-fitted, but the low-level API can be used to update these if need be.
- `transport` takes the prior ensemble `U` and "transports" each realization to a realization of the posterior under the fitted GMRF and `H` model. More on the details here in the low-level API section.

In [ ]:
gtmap = EnIF(Graph_u = Graph_u, Prec_eps=Prec_eps, H=H)
gtmap.fit(U, verbose_level=4)
U_posterior_highlevel = gtmap.transport(U, Y, d, seed=42, verbose_level=10)

In [ ]:
prior_mean = np.array(U.mean(axis=0))
plot_ensemble(U_posterior_highlevel, d, prior_mean)

## EnIF low-level API

### Sanity check #1: Equality of results

We now illustrate the low-level API, and check that it gives the same result as the high-level.

In [ ]:
gtmap_lowlevel = EnIF(Graph_u = Graph_u, Prec_eps=Prec_eps, H=H)

#### Fit
When we call `fit`, we may estimate `Prec_u` as a GMRF w.r.t. `Graph_u`.
Also during `fit`, the mapping `H` may be estimated using `fit_H` that employes the LASSO for learning a sparse representation.

In [ ]:
gtmap_lowlevel.fit_precision(U)
if gtmap_lowlevel.H is None:
    gtmap_lowlevel.fit_H(U, U@H.T) # simulations Y = U@H.T

#### Transport

The name `transport` comes from the field of measure-transport.
It does make sense in this context, because the function indeed transports each realization of prior to a realization of the posterior w.r.t. the GMRF on $(u,d)$ estimated by `fit`.

`transport` consists of three functions
- `pushforward_to_canonical`: transport realizations to "canonical" space
- `update_canonical`: update each canonical realization to a realization from the posterior, _and_ update the precision `Prec_u`
- `pullback_from_canonical`transport realizations back to physical space

where in-between, we sample some specific noisy residuals.

In [ ]:
canonical = gtmap_lowlevel.pushforward_to_canonical(U)

# Work out residuals and associate unexplained variance
residual = gtmap_lowlevel.response_residual(U, Y)
eps = gtmap_lowlevel.generate_observation_noise(n, seed=42)
residual_noisy = residual + eps

canonical_posterior = gtmap_lowlevel.update_canonical(canonical, residual_noisy, d)
U_posterior_lowlevel = gtmap_lowlevel.pullback_from_canonical(canonical_posterior, iterative=True)

In [ ]:
canonical_posterior.shape

Note that `transport` could have used the Kalman-filter equations (conditioning with a multivariate Gaussian in the mean-precision parametrization).
It is however much more numerically efficient (and stable) to employ the Information-filter equations (conditioning with a multivariate Gaussian in the canonical parametrization) when working with a GMRF.

We plot to check the results are like before

In [ ]:
plot_ensemble(U_posterior_lowlevel, d, prior_mean)

The results look identical.

In [ ]:
np.allclose(U_posterior_lowlevel, U_posterior_highlevel)

And indeed they are identical. When `seed` is different, the results should be distributionally equivalent.

### Sanity check #2: Using true precision

For this case, we do indeed have access to the true prior precision of the AR-1 process.
It is possible to pass this to `EnIF` upon initialization, and if `H` too is provided it is not necessary to run `fit`.
The case of a known precision is sometimes known, e.g. when doing pure parameter estimation, the prior we are sampling from can indeed be known, and is often transformed (for good reasons) to a multivariate standard normal.

In [ ]:
gtmap_prec_known = EnIF(Prec_u = Prec_u, Prec_eps=Prec_eps, H=H)

U_posterior = gtmap_prec_known.transport(U, Y, d)

plot_ensemble(U_posterior, d, prior_mean)

The results are _almost_ identical.
Note however that there is zero noise in the update.
This is because we have not estimated anything from data -- both `Prec_u` and `H` was provided apriori, exactly defining the Gaussian on $(u,d)$.

### Sanity check #3: Pullback of pushforward without update at canonical

We just decomposed `transport` to its three main parts, a pushforward, an update, and a pullback.
It is reasonable to think that if we skip the `update` in the middle,

In [ ]:
gtmap_pullpush = EnIF(Prec_u = Prec_u, Prec_eps=Prec_eps, H=H)
canonical = gtmap_pullpush.pushforward_to_canonical(U)
U_posterior = gtmap_pullpush.pullback_from_canonical(canonical)

then the pullback of what we pushforwarded, should be the same as what we input

In [ ]:
np.allclose(U, U_posterior, atol=1e-12)

which indeed seems to be the case.

## Estimating a sparse linear map

If `H` is not given, then it must be estimated from data `U` and `Y`.
We seek a linear map that learns
$$H = E[\nabla_u h(u)]$$ 
for some conceptual potentially non-linear map $h:u\mapsto y$.
Furthermore, we seek a sparse representation `H`, so that the posterior `Prec_u` retains sparsity.
To this end, `fit_H` implements linear L1 regression

In [ ]:
gtmap_uninformed = EnIF(Graph_u=Graph_u, Prec_eps=Prec_eps)

Note that neither `Prec_u` or `H` is passed.
As such, they should both be learnt from data.

In [ ]:
gtmap_uninformed.fit(U, Y, verbose_level=4)

Print the true and the (hopefully sparse) learnt map, to see if it looks correct

In [ ]:
print("Original sparse matrix")
print(H) # true (unknown) map
print("Estimated sparse matrix")
print(gtmap_uninformed.H) # estimated (sparse) map

As we see, the sparsity structure has been identified, and the value is very close to `1`.
We may then employ it for `transport` of the prior ensemble to a posterior ensemble

In [ ]:
U_posterior = gtmap_uninformed.transport(U, Y, d)
plot_ensemble(U_posterior, d, prior_mean)

## Estimating a sparse linear map - Fast with influence adapted boosting

In [ ]:
gtmap_uninformed_2 = EnIF(Graph_u=Graph_u, Prec_eps=Prec_eps)

In [ ]:
gtmap_uninformed_2.fit(U, Y, learning_algorithm="influence-boost", verbose_level=4)

In [ ]:
print("Original sparse matrix")
print(H) # true (unknown) map
print("Estimated sparse matrix")
print(gtmap_uninformed_2.H) # estimated (sparse) map

In [ ]:
U_posterior = gtmap_uninformed_2.transport(U, Y, d)
plot_ensemble(U_posterior, d, prior_mean)